In [7]:
!conda install -c conda-forge arabic-reshaper python-bidi -y

^C


Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000025B160DD010>, 'Connection to conda.anaconda.org timed out. (connect timeout=9.15)')': /conda-forge/terms.json

Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000025B151DA350>, 'Connection to conda.anaconda.org timed out. (connect timeout=9.15)')': /conda-forge/terms.json

Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000025B151DA0D0>, 'Connection to conda.anaconda.org timed out. (connect timeout=9.15)')': /conda-forge/terms.json

Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(

Jupyter detected...
3 channel Terms of Service accepted
Retrieving notices: done
Channels:
 - conda-forge
 - defaults
Platform: win-64


Import کتابخانه‌های مورد نیاز

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import arabic_reshaper
from bidi.algorithm import get_display
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, learning_curve, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

def persian_text(text):
    reshaped = arabic_reshaper.reshape(text)
    return get_display(reshaped)

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
sns.set(style="whitegrid")

ModuleNotFoundError: No module named 'arabic_reshaper'

بارگذاری و خواندن دیتاست

In [ ]:
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['data/data.csv'] = data.target

print(f"تعداد نمونه‌ها: {df.shape[0]}")
print(f"تعداد ویژگی‌ها: {df.shape[1]-1}")
print(f"نام کلاس‌ها: {data.target_names}")
df.head()

اکتشاف کامل دیتاست (EDA)

In [ ]:
# اطلاعات کلی
df.info()

# آمار توصیفی
df.describe()

# توزیع کلاس‌ها
print("توزیع کلاس‌ها:")
print(df['data/data.csv'].value_counts(normalize=True))

plt.figure(figsize=(8, 5))
sns.countplot(x='data/data.csv', data=df)
plt.title(persian_text('توزیع کلاس‌های هدف (0: malignant, 1: benign)'))
plt.show()

# همبستگی ویژگی‌ها با هدف
correlations = df.corr()['data/data.csv'].sort_values(ascending=False)
print("۱۰ ویژگی با بیشترین همبستگی مطلق با هدف:")
print(correlations.abs().sort_values(ascending=False).head(11)[1:])  # بدون خود target

# نمودار پراکندگی چند ویژگی مهم
important_features = ['worst concave points', 'mean concave points', 'worst radius']
sns.pairplot(df, vars=important_features, hue='data/data.csv', diag_kind='kde')
plt.show()

شناسایی Missing Values

In [ ]:
print(f"تعداد مقادیر گمشده: {df.isnull().sum().sum()}")

ترسیم نمودارهای مختلف برای درک بهتر

In [ ]:
# هیستوگرام چند ویژگی مهم
df[important_features].hist(bins=30, figsize=(12, 8))
plt.suptitle('توزیع ویژگی‌های مهم')
plt.show()

# ماتریس همبستگی ویژگی‌های با همبستگی بالا
high_corr_features = correlations.abs().sort_values(ascending=False).head(15).index
plt.figure(figsize=(12, 10))
sns.heatmap(df[high_corr_features].corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('ماتریس همبستگی ویژگی‌های مهم')
plt.show()

پیش‌پردازش و تقسیم داده‌ها

In [ ]:
X = df.drop('data/data.csv', axis=1)
y = df['data/data.csv']

# مقیاس‌بندی (ضروری برای SVM و kNN)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# تقسیم ثابت برای ارزیابی نهایی
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

print(f"اندازه train: {X_train.shape[0]}, test: {X_test.shape[0]}")

تابع کمکی برای ارزیابی با حجم‌های مختلف آموزشی

In [ ]:
def evaluate_with_different_sizes(model, name):
    train_sizes = np.linspace(0.1, 1.0, 10)
    accuracies = []
    
    for size in train_sizes:
        n_samples = int(size * len(X_train))
        X_sub = X_train[:n_samples]
        y_sub = y_train[:n_samples]
        
        model.fit(X_sub, y_sub)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        accuracies.append(acc)
    
    # plot learning curve
    plt.plot(train_sizes, accuracies, marker='o', label=name)
    
    # بهترین مدل روی کل train
    model.fit(X_train, y_train)
    y_pred_final = model.predict(X_test)
    print(f"\nنتایج نهایی {name} (روی کل داده آموزشی):")
    print(f"Accuracy: {accuracy_score(y_test, y_pred_final):.4f}")
    print(classification_report(y_test, y_pred_final, target_names=data.target_names))
    
    return model

آموزش و ارزیابی مدل‌ها

In [ ]:
plt.figure(figsize=(12, 8))
plt.title('منحنی یادگیری (Accuracy بر حسب درصد داده آموزشی)')
plt.xlabel('درصد داده آموزشی')
plt.ylabel('Accuracy روی تست')

# ۱. Logistic Regression
log_reg = LogisticRegression(max_iter=10000)
evaluate_with_different_sizes(log_reg, 'Logistic Regression')

# ۲. Linear SVM
linear_svm = SVC(kernel='linear')
evaluate_with_different_sizes(linear_svm, 'Linear SVM')

# ۳. Non-linear SVM (RBF)
rbf_svm = SVC(kernel='rbf', gamma='scale')
evaluate_with_different_sizes(rbf_svm, 'RBF SVM')

# ۴. kNN (با انتخاب بهترین k)
param_grid = {'n_neighbors': range(3, 20)}
knn_grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy')
knn_grid.fit(X_train, y_train)
best_knn = knn_grid.best_estimator_
print(f"بهترین k: {knn_grid.best_params_['n_neighbors']}")
evaluate_with_different_sizes(best_knn, f'kNN (k={knn_grid.best_params_["n_neighbors"]})')

plt.legend()
plt.grid(True)
plt.show()

Confusion Matrix

In [ ]:
models = {
    'Logistic Regression': log_reg,
    'Linear SVM': linear_svm,
    'RBF SVM': rbf_svm,
    'kNN': best_knn
}

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.ravel()

for idx, (name, model) in enumerate(models.items()):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    ConfusionMatrixDisplay.from_predictions(y_test, y_pred, display_labels=data.target_names, ax=axes[idx])
    axes[idx].set_title(name)

plt.tight_layout()
plt.show()